# Khai báo các thư viện

In [68]:
import pandas as pd
from datetime import datetime

# Lấy dữ liệu khách hàng từ file csv

In [69]:
df = pd.read_csv("sample-orders.csv", sep=',', encoding="windows-1252")

chuyển đổi dữ liệu ngày tháng từ dạng chuỗi thành datetime object

In [70]:
df['ngay_dat_mua'] = pd.to_datetime(df['ngay_dat_mua'])
NOW = datetime.strptime("2022-12-31", "%Y-%m-%d")

# Tính các giá trị R, F, M từ dữ liệu khách hàng

In [71]:
bang_rfm = df.groupby('khach_hang').agg({'ngay_dat_mua': lambda x: (NOW - x.max()).days,  # Recency
                                         'ma_mua_hang': lambda x: len(x),  # Frequency
                                         'tong_gia_tri_hang': lambda x: x.sum()})

bang_rfm.rename(columns={'ngay_dat_mua': 'recency',
                         'ma_mua_hang': 'frequency',
                         'tong_gia_tri_hang': 'monetary_value'}, inplace=True)

bang_rfm.head()

,recency,frequency,monetary_value
khach_hang,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


# Quy đổi giá trị R, F, M theo thang đo từ 1 đến 5
Tính ngũ phân vị của R, F, M

In [72]:
ngu_phan_vi = bang_rfm.quantile(q=[0.2, 0.4, 0.6, 0.8])
print(ngu_phan_vi)
ngu_phan_vi = ngu_phan_vi.to_dict()

     recency  frequency  monetary_value
0.2     26.0        4.0           961.4
0.4     53.0        5.0          1739.8
0.6    103.0        7.0          2750.8
0.8    224.6        8.6          4282.6


Tạo hàm phân loại R, F, M theo thang 5

In [73]:
def ngu_phan_vi_recency(x, p, d):
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]:
        return 3
    elif x <= d[p][0.8]:
        return 2
    else:
        return 1

def ngu_phan_vi_frequency_monetary(x, p, d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]:
        return 3
    elif x <= d[p][0.8]:
        return 4
    else:
        return 5

Tiến hành phân đoạn và đưa kết quả vào cột mới

In [74]:
phan_doan_khach_hang = bang_rfm

phan_doan_khach_hang['ngu_phan_vi_R'] = phan_doan_khach_hang['recency'].apply(ngu_phan_vi_recency, args=('recency', ngu_phan_vi,))

phan_doan_khach_hang['ngu_phan_vi_F'] = phan_doan_khach_hang['frequency'].apply(ngu_phan_vi_frequency_monetary, args=('frequency', ngu_phan_vi,))

phan_doan_khach_hang['ngu_phan_vi_M'] = phan_doan_khach_hang['monetary_value'].apply(ngu_phan_vi_frequency_monetary, args=('monetary_value', ngu_phan_vi,))

Tổng hợp kết quả lại thành điểm RFM và xuất dữ liệu vào file output.csv

In [75]:
phan_doan_khach_hang['Diem_RFM'] = phan_doan_khach_hang.ngu_phan_vi_R.map(str) + \
                                           phan_doan_khach_hang.ngu_phan_vi_F.map(str) + \
                                           phan_doan_khach_hang.ngu_phan_vi_M.map(str)

    # Ghi bảng phan_doan_khach_hang vào tệp CSV outputfile với dấu phân cách ,
phan_doan_khach_hang.to_csv("output.csv")